In [69]:
import pandas as pd
import datetime


In [70]:
def parseDatetime(x: str):
    rus = {"янв": "jan",
           "февр": "feb", "фев": "feb",
           "мар": "mar",
           "апр": "apr",
           "май": "may", "мая": "may",
           "июн": "jun",
           "июл": "jul",
           "авг": "aug",
           "сент": "sep", "сен": "sep",
           "окт": "oct",
           "нояб": "nov", "ноя": "nov",
           "дек": "dec",
           "г. ": "", 
           ".": ""}
    for r, e in rus.items():
        x = x.lower().replace(r, e)
    try:
        dt = datetime.datetime.strptime(x.lower(), u'%d %b %Y %H:%M:%S')
    except:
        dt = datetime.datetime.strptime(x.lower(), u'%d%m%Y %H:%M:%S')
    return dt


In [71]:
df = pd.read_csv("data/test.csv", on_bad_lines="warn", sep="\t")
df = df.drop(columns=['Project', 'Account',
             'Payment account', 'Merchant', 
                      'Address', 'Note', 'Tags', 'Author', 'Image 1', 'Image 2',
                      'Image 3'])
df['Date time'] = df['Date time'].apply(lambda x: parseDatetime(x))



In [72]:
expense = df[df["Type"] == "Expense"]
income = df[df["Type"] == "Income"]
invest = df[(df["Type"] == "Transfer") & (df["Category"] == "Investing")]

